In [156]:
# package imports
#basics
import numpy as np
import pandas as pd

#viz
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.gridspec as gridspec 
import matplotlib.gridspec as gridspec 


#settings

color = sns.color_palette()
sns.set_style("dark")
pd.set_option('display.max_columns', None)

%matplotlib inline

In [157]:
yelp=pd.read_csv('final_yelp.csv')

In [158]:
yelp.head(2)

,id,alias,name,image_url,is_closed,url,review_count,category alias,category title,rating,coordinates,latitude,longitude,street,apt,area,city,pincode,country,state,address1,address2,phone,display_phone,distance,price
0,33tYNCEBSlDaQOgRq8-L9g,muse-bar-new-york,Muse Bar,https://s3-media2.fl.yelpcdn.com/bphoto/JXlRGp...,False,https://www.yelp.com/biz/muse-bar-new-york?adj...,6,cocktailbars,Cocktail Bars,5.0,40.757920,-73.983750,NaN,130 W 46th St,NaN,None,New York,10036.00,US,NY,130 W 46th St-New York,NY 10036,12124852999,(212) 485-2999,520.1101292,
1,rVwQcrogYw-O2SsAk51iYA,bemelmans-bar-new-york,Bemelmans Bar,https://s3-media2.fl.yelpcdn.com/bphoto/rllg-W...,False,https://www.yelp.com/biz/bemelmans-bar-new-yor...,302,bars,Bars,4.0,40.774379,-73.963308,NaN,35 E 76th St,NaN,Carlyle Hotel,New York,10021.00,US,NY,35 E 76th St-Carlyle Hotel-New York,NY 10021,12127441600,(212) 744-1600,2029.144814,$$$


In [159]:
# Taking only the required columns

In [160]:
yelp=yelp[['id','name','is_closed','review_count','category alias','category title','rating','coordinates',
          'latitude','pincode','address2','distance','price']]

In [161]:
yelp.head(1)

,id,name,is_closed,review_count,category alias,category title,rating,coordinates,latitude,pincode,address2,distance,price
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.75792,-73.98375,10036.00,NY 10036,520.1101292,


In [162]:
yelp.isnull().sum()

id                0
name              0
is_closed         0
review_count      0
category alias    0
category title    0
rating            0
coordinates       0
latitude          0
pincode           4
address2          0
distance          0
price             1
dtype: int64

In [163]:
yelp.dropna(subset=['pincode','price'],inplace=True)

In [164]:
yelp.isnull().sum()

id                0
name              0
is_closed         0
review_count      0
category alias    0
category title    0
rating            0
coordinates       0
latitude          0
pincode           0
address2          0
distance          0
price             0
dtype: int64

In [165]:
#yelp['pincode'] = yelp['pincode'].astype(int)

In [166]:
yelp['zipcode']=yelp['address2'].str.split(' ').str[2]

In [167]:
yelp.head(1)

,id,name,is_closed,review_count,category alias,category title,rating,coordinates,latitude,pincode,address2,distance,price,zipcode
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.75792,-73.98375,10036.00,NY 10036,520.1101292,,10036


In [168]:
yelp.dtypes

id                 object
name               object
is_closed            bool
review_count        int64
category alias     object
category title     object
rating            float64
coordinates       float64
latitude          float64
pincode            object
address2           object
distance           object
price              object
zipcode            object
dtype: object

In [169]:
yelp['zipcode'] = yelp['zipcode'].astype(int)

In [170]:
yelp.rename(columns = {'coordinates':'latitude', 'latitude':'longitude'}, inplace = True) 

In [171]:
yelp.drop(['pincode','address2'],axis=1,inplace=True)

In [172]:
yelp.head(2)

,id,name,is_closed,review_count,category alias,category title,rating,latitude,longitude,distance,price,zipcode
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.757920,-73.983750,520.1101292,,10036
1,rVwQcrogYw-O2SsAk51iYA,Bemelmans Bar,False,302,bars,Bars,4.0,40.774379,-73.963308,2029.144814,$$$,10021


In [173]:
yelp.shape

(8457, 12)

In [174]:
# Recode price as int
yelp['price'] = yelp['price'].map({'$':1, "$$":2, '$$$':3, '$$$$':4})

In [175]:
yelp.head(2)

,id,name,is_closed,review_count,category alias,category title,rating,latitude,longitude,distance,price,zipcode
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.757920,-73.983750,520.1101292,NaN,10036
1,rVwQcrogYw-O2SsAk51iYA,Bemelmans Bar,False,302,bars,Bars,4.0,40.774379,-73.963308,2029.144814,3.0,10021


In [176]:
# Build borough column
zip_to_bor = pd.read_csv('neighborhood-NYC.csv')
borough_dict = dict(zip(zip_to_bor['zipcode'], zip_to_bor['borough']))
yelp['borough'] = yelp['zipcode'].map(borough_dict)

In [177]:
yelp.borough.value_counts()

Manhattan        2767
Brooklyn         2025
Queens           1471
Bronx             766
Staten Island     509
New Jersey        367
Name: borough, dtype: int64

In [178]:
yelp.isnull().sum()

id                   0
name                 0
is_closed            0
review_count         0
category alias       0
category title       0
rating               0
latitude             0
longitude            0
distance             0
price             1534
zipcode              0
borough            552
dtype: int64

In [179]:
# yelp[yelp['borough'].isnull() & (yelp['neighborhood'].isnull())]['zipcode'].value_counts()
# yelp[yelp['borough'].isnull()].index.tolist()
# On inspecting, I found out that these zipcodes were either from NJ or some random, 
# some restaurants other than NYC were fecthed while calling the API 

In [180]:
yelp.dropna(subset=['borough',],inplace=True)

In [181]:
# Building neighborhood column based on neighborhood group of Airbnb

In [182]:
borough_dict1 = dict(zip(zip_to_bor['zipcode'], zip_to_bor['neighborhood']))
yelp['neighborhood'] = yelp['zipcode'].map(borough_dict1)

In [183]:
yelp.head(2)

,id,name,is_closed,review_count,category alias,category title,rating,latitude,longitude,distance,price,zipcode,borough,neighborhood
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.757920,-73.983750,520.1101292,NaN,10036,Manhattan,Chelsea and Clinton
1,rVwQcrogYw-O2SsAk51iYA,Bemelmans Bar,False,302,bars,Bars,4.0,40.774379,-73.963308,2029.144814,3.0,10021,Manhattan,Upper East Side


In [192]:
yelp.isnull().sum()

id                0
name              0
is_closed         0
review_count      0
category alias    0
category title    0
rating            0
latitude          0
longitude         0
distance          0
price             0
zipcode           0
borough           0
neighborhood      0
dtype: int64

In [186]:
yelp['price'].median()

2.0

In [187]:
# replacing Price with median

In [190]:
yelp=yelp.fillna(yelp.median())

In [193]:
yelp.to_csv('yelp_cleaned.csv')